## Data Preparation and Exploration

This notebook is focused on preparing, cleaning and exploring a dataset that as been gathered at two solar power plants in India over a 34 day period

The goal is to apply this dataset to machine learning models to predict solar power plant output.


Points to look out for in a clean data:
* Making sure the data is presented in a way that makes sense with all the variables correct.
* Ensure that the variables make sense and are correct.
* Eliminating duplicate rows and columns as well as elimiating columns that provide redundant information.
* Dealing with missing data appropriately.

In [142]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import warnings
warnings.filterwarnings("ignore")


### Preliminary Analysis
The power generation datasets are gathered at the inverter level - each inverter has multiple lines of solar panels attached to it. The sensor data is gathered at a plant level - single array of sensors optimally placed at the plant.

In [143]:
### Reading in the data
generation_dataset = pd.read_csv('Plant_1_Generation_Data.csv')
weather_dataset = pd.read_csv('Plant_2_Weather_Sensor_Data.csv')

In [144]:
generation_dataset.head(10)

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0
5,15-05-2020 00:00,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0
6,15-05-2020 00:00,4135001,WRmjgnKYAwPKWDb,0.0,0.0,0.0,7028673.0
7,15-05-2020 00:00,4135001,ZnxXDlPa8U1GXgE,0.0,0.0,0.0,6522172.0
8,15-05-2020 00:00,4135001,ZoEaEvLYb1n2sOq,0.0,0.0,0.0,7098099.0
9,15-05-2020 00:00,4135001,adLQvlD726eNBSB,0.0,0.0,0.0,6271355.0


In [145]:
generation_dataset.shape

(68778, 7)

Generation dataset contains 68778 rows and 7 columns

In [146]:
weather_dataset.head(10)

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4136001,iq8k7ZNt4Mwm3w0,27.004764,25.060789,0.0
1,2020-05-15 00:15:00,4136001,iq8k7ZNt4Mwm3w0,26.880811,24.421869,0.0
2,2020-05-15 00:30:00,4136001,iq8k7ZNt4Mwm3w0,26.682055,24.427290,0.0
3,2020-05-15 00:45:00,4136001,iq8k7ZNt4Mwm3w0,26.500589,24.420678,0.0
4,2020-05-15 01:00:00,4136001,iq8k7ZNt4Mwm3w0,26.596148,25.088210,0.0
5,2020-05-15 01:15:00,4136001,iq8k7ZNt4Mwm3w0,26.512740,25.317970,0.0
6,2020-05-15 01:30:00,4136001,iq8k7ZNt4Mwm3w0,26.494339,25.217193,0.0
7,2020-05-15 01:45:00,4136001,iq8k7ZNt4Mwm3w0,26.420410,25.065062,0.0
8,2020-05-15 02:00:00,4136001,iq8k7ZNt4Mwm3w0,26.401946,24.691469,0.0
9,2020-05-15 02:15:00,4136001,iq8k7ZNt4Mwm3w0,26.226078,24.559481,0.0


In [147]:
weather_dataset.shape

(3259, 6)

Weather dataset contains 3259 rows and 6 columns

Formatting DATE_TIME columns and merging the two dataframes.

In [148]:
generation_dataset['DATE_TIME'] = pd.to_datetime(generation_dataset['DATE_TIME'])
weather_dataset['DATE_TIME'] = pd.to_datetime(weather_dataset['DATE_TIME'])

In [149]:
df_solar_data = pd.merge(generation_dataset.drop(columns = ['PLANT_ID']), weather_dataset.drop(columns = ['PLANT_ID', 'SOURCE_KEY']), on='DATE_TIME')

In [150]:
# A summary of each column, the datatypes and the number of non-null values 

df_solar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68708 entries, 0 to 68707
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_TIME            68708 non-null  datetime64[ns]
 1   SOURCE_KEY           68708 non-null  object        
 2   DC_POWER             68708 non-null  float64       
 3   AC_POWER             68708 non-null  float64       
 4   DAILY_YIELD          68708 non-null  float64       
 5   TOTAL_YIELD          68708 non-null  float64       
 6   AMBIENT_TEMPERATURE  68708 non-null  float64       
 7   MODULE_TEMPERATURE   68708 non-null  float64       
 8   IRRADIATION          68708 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 4.7+ MB


Convert 'SOURCE_KEY' to numerical type

In [151]:
from sklearn.preprocessing import *

encoder = LabelEncoder()

df_solar_data['SOURCE_KEY_NUMBER'] = encoder.fit_transform(df_solar_data['SOURCE_KEY'])

df_solar_data.drop(columns=['SOURCE_KEY'], inplace=True)

In [152]:
df_solar_data.sample(10).style.background_gradient(cmap='hot')

,DATE_TIME,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY_NUMBER
13403,2020-05-22 03:00:00,0.000000,0.000000,0.000000,7057147.000000,24.281303,22.438980,0.000000,19
25031,2020-05-27 19:45:00,0.000000,0.000000,6638.000000,7698550.000000,31.482985,29.895664,0.000000,3
38381,2020-06-03 12:45:00,12406.571430,1210.357143,3995.857143,6325306.857000,29.988368,38.625959,0.274321,13
50171,2020-06-09 04:45:00,0.000000,0.000000,0.000000,7281724.000000,22.857227,20.404605,0.000000,9
38956,2020-06-03 19:30:00,0.000000,0.000000,7846.000000,7163293.000000,25.206124,23.932733,0.000000,16
45464,2020-06-06 23:15:00,0.000000,0.000000,6603.000000,6445031.000000,24.732901,23.273788,0.000000,10
51790,2020-06-09 23:15:00,0.000000,0.000000,7456.000000,6439070.000000,25.759716,24.364314,0.000000,0
18396,2020-05-24 14:30:00,9616.250000,939.112500,6416.750000,7240091.750000,37.261042,56.476959,0.729815,14
49570,2020-06-08 22:00:00,0.000000,0.000000,8509.000000,7175733.000000,26.792292,25.278673,0.000000,2
30222,2020-05-30 15:30:00,8288.875000,811.212500,6268.125000,6642334.125000,36.871172,48.761178,0.575536,8


### Looking for null and missing values.

In [153]:
df_solar_data.isnull().sum()


DATE_TIME              0
DC_POWER               0
AC_POWER               0
DAILY_YIELD            0
TOTAL_YIELD            0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
SOURCE_KEY_NUMBER      0
dtype: int64

### Checking for Duplicate rows/ redundant information

In [154]:
#Checking for duplicate rows
df_solar_data.duplicated().sum()

0

In [155]:
#Checking for duplicate columns
df_solar_data.T.duplicated()

DATE_TIME              False
DC_POWER               False
AC_POWER               False
DAILY_YIELD            False
TOTAL_YIELD            False
AMBIENT_TEMPERATURE    False
MODULE_TEMPERATURE     False
IRRADIATION            False
SOURCE_KEY_NUMBER      False
dtype: bool